In [2]:
import pandas as pd
import numpy as np

In [4]:
# read csv exported from streamlit
df = pd.read_csv('uk_sales_2012_sample.csv')

In [6]:
df.dtypes

Invoice        float64
StockCode      float64
Description     object
Quantity       float64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
Gender          object
Location        object
Age            float64
dtype: object

# Preprocessing

In [7]:
# change InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [8]:
df.dtypes

Invoice               float64
StockCode             float64
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
Gender                 object
Location               object
Age                   float64
dtype: object

# RFM

In [9]:
max_date = df['InvoiceDate'].max()
recency_df = df.groupby('Customer ID')['InvoiceDate'].max().reset_index()
recency_df['Recency'] = (max_date - recency_df['InvoiceDate']).dt.days
recency_df = recency_df[['Customer ID', 'Recency']]

frequency_df = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
frequency_df.rename(columns={'Invoice': 'Frequency'}, inplace=True)

df['MonetaryValue'] = (df['Price'] * df['Quantity']).round(2)
monetary_df = df.groupby('Customer ID')['MonetaryValue'].sum().round(2).reset_index()
monetary_df.rename(columns={'MonetaryValue': 'Monetary'}, inplace=True)

customer_info = df.groupby('Customer ID').first()[['Gender', 'Location', 'Age']].reset_index()

rfm_df = recency_df.merge(frequency_df, on='Customer ID') \
                   .merge(monetary_df, on='Customer ID') \
                   .merge(customer_info, on='Customer ID')


In [11]:
rfm_df.head()

,Customer ID,Recency,Frequency,Monetary,Gender,Location,Age
0,10001.0,0,3,762.90,Male,York,69.0
1,10002.0,105,1,82.64,Female,Durham,32.0


# Top locations

In [21]:
# top_locations = 
top_locations = df.groupby('Location')['Invoice'].nunique().reset_index()

# Sắp xếp theo số lượng hóa đơn xuất hiện nhiều nhất
top_locations = top_locations.sort_values(by='Invoice', ascending=False)


In [22]:
top_locations

,Location,Invoice
1,York,3
0,Durham,1


# Top customers

In [23]:
top_customers = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
top_customers = top_customers.sort_values(by='Invoice', ascending=False)

In [24]:
top_customers

,Customer ID,Invoice
0,10001.0,3
1,10002.0,1


# Revenue

In [44]:
from questions_classification import QuestionClassifierModel

classifier = QuestionClassifierModel.load_model("question_classifier.pkl")

question = "RFM clustering"
predicted_category = classifier.predict(question)

print(f"Predicted Category: {predicted_category}")


Predicted Category: RFM Clustering


In [45]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

max_date = df['InvoiceDate'].max()
recency_df = df.groupby('Customer ID')['InvoiceDate'].max().reset_index()
recency_df['Recency'] = (max_date - recency_df['InvoiceDate']).dt.days
recency_df = recency_df[['Customer ID', 'Recency']]

frequency_df = df.groupby('Customer ID')['Invoice'].nunique().reset_index()
frequency_df.rename(columns={'Invoice': 'Frequency'}, inplace=True)

df['MonetaryValue'] = (df['Price'] * df['Quantity']).round(2)
monetary_df = df.groupby('Customer ID')['MonetaryValue'].sum().round(2).reset_index()
monetary_df.rename(columns={'MonetaryValue': 'Monetary'}, inplace=True)

customer_info = df.groupby('Customer ID').first()[['Gender', 'Location', 'Age']].reset_index()

rfm_df = recency_df.merge(frequency_df, on='Customer ID') \
                   .merge(monetary_df, on='Customer ID') \
                   .merge(customer_info, on='Customer ID')

In [46]:
rfm_df

,Customer ID,Recency,Frequency,Monetary,Gender,Location,Age
0,10001.0,0,3,762.90,Male,York,69.0
1,10002.0,105,1,82.64,Female,Durham,32.0
